<a href="https://colab.research.google.com/github/NamanNYC/Final_Project/blob/database4/db_write.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()  

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [67.9 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 h

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-25 15:43:37--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-09-25 15:43:37 (7.29 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
# Start Spark session
from pyspark.sql import SparkSession
from pyspark.sql.functions import substring, length, col, expr, to_timestamp, date_format, round

spark = SparkSession.builder.appName("LMPT-Forest-Fires").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
connection_string = 'lmpt-finalproject.coke2w4vs8wf.us-east-2.rds.amazonaws.com'
password = 'LMPTp4ssw0rd' 
database_name = 'postgres'

# Configure settings for RDS
mode = "append"
jdbc_url=f"jdbc:postgresql://{connection_string}:5432/{database_name}"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}


In [11]:
# Read in data 
df = spark.read.jdbc(jdbc_url,table='fires_2006to2018',properties=config)

In [24]:
df.dtypes

[('fire_number', 'string'),
 ('fire_name', 'string'),
 ('fire_year', 'int'),
 ('calendar_year', 'int'),
 ('assessment_datetime', 'timestamp'),
 ('assessment_hectares', 'double'),
 ('current_size', 'double'),
 ('size_class', 'string'),
 ('fire_location_latitude', 'double'),
 ('fire_location_longitude', 'double'),
 ('fire_origin', 'string'),
 ('general_cause_desc', 'string'),
 ('industry_identifier_desc', 'string'),
 ('responsible_group_desc', 'string'),
 ('activity_class', 'string'),
 ('true_cause', 'string'),
 ('permit_detail_desc', 'string'),
 ('fire_start_date', 'timestamp'),
 ('det_agent_type', 'string'),
 ('det_agent', 'string'),
 ('discovered_date', 'timestamp'),
 ('reported_date', 'timestamp'),
 ('start_for_fire_date', 'timestamp'),
 ('fire_fighting_start_date', 'string'),
 ('fire_fighting_start_size', 'string'),
 ('initial_action_by', 'string'),
 ('fire_type', 'string'),
 ('fire_position_on_slope', 'string'),
 ('weather_conditions_over_fire', 'string'),
 ('fuel_type', 'string'),

In [23]:
from pyspark.sql.functions import to_timestamp, date_format
df = df.withColumn('fire_start_date', to_timestamp (col('fire_start_date')))
df = df.withColumn('assessment_datetime', to_timestamp (col('assessment_datetime')))
df = df.withColumn('discovered_date', to_timestamp (col('discovered_date')))
df = df.withColumn('reported_date', to_timestamp (col('reported_date')))
df = df.withColumn('start_for_fire_date', to_timestamp (col('start_for_fire_date')))



In [26]:
detection = df[['fire_number','industry_identifier_desc','det_agent_Type','det_agent','discovered_date','reported_date']] 
loc = df[['fire_number','fire_location_latitude','fire_location_longitude']]
cause = df[['fire_number','responsible_group_desc','activity_class','true_cause','permit_detail_desc','fire_start_date']]
response = df[['fire_number','start_for_fire_date','fire_fighting_start_size','initial_action_by','bh_fs_date','bh_hectares','uc_fs_date','uc_hectares']]

In [25]:
detection.write.jdbc(url=jdbc_url, table='detection', mode=mode, properties=config)
loc.write.jdbc(url=jdbc_url, table='loc', mode=mode, properties=config)
cause.write.jdbc(url=jdbc_url, table='cause', mode=mode, properties=config)



In [27]:
response.write.jdbc(url=jdbc_url, table='Response', mode=mode, properties=config)


Py4JJavaError: ignored